In [6]:
# Fix for issue loading Utils.preprocess_util
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from Utils.preprocess_util import *
from Utils.visualize import *

In [2]:
X_train,X_valid,X_test,Y_train,Y_valid,Y_test = load_preprocess_eeg_data()

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)

Training data shape: (1417, 22, 1000)
Valid data shape: (698, 22, 1000)
Training target shape: (1417,)
Valid target shape: (698,)


In [3]:
# create feature and targets tensor for train set.  we create tensor, then we will create variable
features_train = torch.from_numpy(X_train)
targets_train = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
features_test = torch.from_numpy(X_test)
targets_test = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

features_valid = torch.from_numpy(X_valid)
targets_valid = torch.from_numpy(Y_valid).type(torch.LongTensor)

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(LSTMModel, self).__init__()
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        # 2 is the number of lstms stacked vertically(num_layers)
        self.lstm = nn.LSTM(self.n_inputs, self.n_neurons,2) 
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
            # (num_layers, batch_size, n_neurons)
            # 2 is the number of lstms stacked vertically (num_layers)
            return (torch.zeros(2, self.batch_size, self.n_neurons))

    def forward(self, X):
            # transforms X to (n_steps, batch_size, n_inputs)
            X = X.permute(1, 0, 2) 
            self.batch_size = X.size(1)
            self.hidden = self.init_hidden()
            self.cellstate = self.init_hidden()
            lstm_out, (self.hidden, self.cellstate)= self.lstm(X, (self.hidden,self.cellstate))      
            out = self.FC(self.hidden[1])

            return out.view(-1, self.n_outputs) # (batch_size, n_output)


In [7]:
batch_size = 500
# Pytorch train and test sets
train = torch.utils.data.TensorDataset(features_train, targets_train)
valid = torch.utils.data.TensorDataset(features_valid, targets_valid)
test = torch.utils.data.TensorDataset(features_test, targets_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)


In [8]:
N_STEPS = 1000
N_INPUTS = 22
N_NEURONS = 10
N_OUTPUTS = 10
N_EPOCHS = 10

dataiter = iter(train_loader)
images, labels = dataiter.next()
model = LSTMModel(batch_size, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)

#batch_size X n_steps X  n_inputs
images_modified = images.view(-1, 1000,22)
logits = model(images_modified.float())
print(logits[0:10])

tensor([[-0.1506, -0.0272, -0.2063, -0.2443,  0.1812,  0.0147,  0.0365,  0.1577,
          0.0164, -0.1232],
        [-0.1281, -0.0307, -0.1909, -0.2324,  0.1653, -0.0393,  0.0362,  0.1471,
          0.0013, -0.1673],
        [-0.1266, -0.0197, -0.1972, -0.2239,  0.1868, -0.0210,  0.0336,  0.1649,
         -0.0016, -0.1685],
        [-0.1295, -0.0172, -0.2084, -0.2242,  0.1873, -0.0284,  0.0344,  0.1481,
         -0.0036, -0.1567],
        [-0.1535, -0.0199, -0.2010, -0.2267,  0.1741, -0.0309,  0.0479,  0.1515,
          0.0078, -0.1502],
        [-0.0892, -0.0138, -0.1986, -0.2219,  0.2063, -0.0202,  0.0035,  0.1816,
         -0.0329, -0.1904],
        [-0.1410, -0.0188, -0.2010, -0.2296,  0.1820, -0.0138,  0.0327,  0.1509,
          0.0025, -0.1535],
        [-0.1181, -0.0075, -0.1961, -0.2035,  0.1986, -0.0325,  0.0243,  0.1628,
         -0.0173, -0.1855],
        [-0.1451, -0.0315, -0.1977, -0.2294,  0.1638, -0.0336,  0.0378,  0.1411,
          0.0152, -0.1580],
        [-0.1717, -

In [9]:
dtype = torch.FloatTensor
n_iters = 10000
num_epochs = int(n_iters / (len(X_train)/batch_size))

model = LSTMModel(batch_size, N_STEPS, N_INPUTS, N_NEURONS, N_OUTPUTS)

# Cross Entropy Loss 
loss_fn = nn.CrossEntropyLoss().type(dtype)

# batch GD
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, amsgrad=False)

# Create RNN
input_dim = 22
seq_dim = 1000

train_loss = []
iterations = []
train_acc = []

count = 0
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        train  = Variable(signals.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
        
        # Clear gradients
        optimizer.zero_grad()
        
        # reset hidden states
        model.hidden = model.init_hidden() 
                
        # Forward propagation
        outputs = model(train.float())
        
        # Calculate softmax and cross entropy loss
        loss = loss_fn(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
                    
        #print("parameters===",list(model.parameters())[0].data)

        count += 1
        if count % 1 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for signals, labels in valid_loader:
                signals = Variable(signals.view(-1, seq_dim, input_dim))
                #print(signals.shape)
                # Forward propagation
                outputs_valid = model(signals.float())

                # Get predictions from the maximum value
                predicted = torch.max(outputs_valid.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                correct += (predicted == labels).sum()
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            train_loss.append(loss.data)
            iterations.append(count)
            train_acc.append(accuracy)
            if count % 1 == 0:
                print('Iteration: {}  Loss: {}  Valid Accuracy: {} %'.format(count, loss.data, accuracy))


Iteration: 1  Loss: 2.2869813442230225  Valid Accuracy: 1 %


KeyboardInterrupt: 

In [10]:
X_valid_tensor = torch.from_numpy(X_valid.reshape(-1, seq_dim, input_dim))
print(X_valid_tensor.shape)
y_pred_valid = model( X_valid_tensor.float())
val_acc = get_accuracy(y_pred_valid, Y_valid,
    batch_size=X_valid.shape[0])
print('validation accuracy:', val_acc)


torch.Size([698, 1000, 22])
1.4326647564469914


In [11]:
X_test_tensor = torch.from_numpy(X_test.reshape(-1, seq_dim, input_dim))
y_pred_test = model( X_test_tensor.float())
test_acc = get_accuracy(y_pred_test, Y_test,
    batch_size=X_test.shape[0])
print('test accuracy:', test_acc)


2.4830699774266365
